<a href="https://colab.research.google.com/github/mzucali/pyMinColab/blob/main/ExtractFromSummaryEMPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LEGGE IL FILE SUMMARY.TXT ed estrae DL e SD:

=>>input il file summary.txt o qualunque altro nome MA .TXT
- start Marzo 2023
- aggiornamento Novembre 2023

In [1]:
# load and read summary file from EMPA
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 4-4-23.txt to 4-4-23.txt
User uploaded file "4-4-23.txt" with length 31803 bytes


In [2]:
num_elements = input("QUANTI ELEMENTI ANALIZZATI? ")
#num_elements = 11
loop = int(num_elements)+2

QUANTI ELEMENTI ANALIZZATI? 10


Estrazione SD

In [39]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 14 11:30:45 2023
GRAZIE ALESSANDRO COMUNIAN
@author: alex
"""

#file_in = open("summary.txt", "r")
file_in = open(fn,"r")
outFile = open("result2_tmpSD.txt", "w")

lines = file_in.readlines()

for i, line in enumerate(lines):
    # Just a spacing line
    #line_sp = line.split()
    #print(line_sp)
    line_s = line.split("\t")
    #print(line_s)
    if line_s:
        try:
            if(line_s[2] == "Comment :"):
                #print("line_s[2]",line_s[2])
                # Keep track of the comment value
                comm = line_s[3]
                # Just a spacing line
                #print("comm",comm)
        except IndexError:
            pass

        if(line_s[0] == "Curr.(A) :"):
#        if(line_s[0] == "Element	"):
            #print("ID {0} Comment".format(lines[i+1][:-1]))
            outFile.write("ID {0} Comment\n".format(lines[i+1][:-1]))
            for j in range(2,loop):
                #print("{0} {1}\n".format(lines[i+j][:-1], comm), end="")
                outFile.write("{0} {1}\n".format(lines[i+j][:-1], comm))
                #outFile.write(" {0}\n".format(lines))

file_in.close()
outFile.close() #result2.txt

print("completed SD extraction")

completed SD extraction


Remove spaces SD

In [40]:
#RESULT3 con spazi e tab corretti SD
newopen = open("result2_final_tmpSD.txt", "w")
with open("result2_tmpSD.txt") as f:
  first_line = f.readline()
  first_line_split = first_line.split()
  first_line = '\t'.join(first_line_split)+'\n'
  newopen.write(first_line)
  #print(first_line)

with open("result2_tmpSD.txt") as f:
  for line in f:
    if 'ID Element	Peak(mm)' not in line:
      line_split = line.split()
      line = '\t'.join(line_split)+'\n'
      #print(line)
      newopen.write(line)
newopen.close()


input_file_path = 'result2_final_tmpSD.txt'
output_file_path = 'result2_finalSD.txt'

# Read the file and filter out empty lines
with open(input_file_path, 'r') as file:
    lines = file.readlines()
    non_empty_lines = [line.strip() for line in lines if line.strip()]

# Write non-empty lines back to the file
with open(output_file_path, 'w') as file:
    file.write('\n'.join(non_empty_lines))


os.remove(input_file_path)
os.remove('result2_tmpSD.txt')


print("completed SD")

completed


Estrazione OX

In [41]:
#file_in = open("summary.txt", "r")
file_in = open(fn,"r")
outFile = open("result2_tmpOx.txt", "w")

lines = file_in.readlines()

for i, line in enumerate(lines):
    # Just a spacing line
    #line_sp = line.split()
    #print(line_sp)
    line_s = line.split("\t")
    #line_s.index
    #print("line_s1",line_s)
    if line_s:
        try:
            if(line_s[2] == "Comment :"):
                #print("line_s[2]",line_s[2])
                # Keep track of the comment value
                comm = line_s[3]
                # Just a spacing line
                #print("comm",comm)
        except IndexError:
            pass
        #print("line_s[0]",line_s[2])
        #print("line ",line)
        if len(line_s) == 1:
          pass
        else:
          if(line_s[1] == "El."):
            #print("lines[i][:-1] ",lines[i][:-1])
            #print("ID {0} Comment".format(lines[i][:-1]))
            outFile.write("ID {0} Comment\n".format(lines[i][:-1]))
            for j in range(2,loop+1): #loop+1 perchè non estrae un elemento
              #print("{0} {1}\n".format(lines[i+j][:-1], comm), end="")
              outFile.write("{0} {1}\n".format(lines[i+j][:-1], comm))
              #outFile.write(" {0}\n".format(lines))

file_in.close()
outFile.close() #result2.txt

print("completed OX extraction")

completed OX extraction


In [42]:
#RESULT3 con spazi e tab corretti OX
newopen = open("result2_finalTmpOx.txt", "w")
with open("result2_tmpOx.txt") as f:
  first_line = f.readline()
  first_line_split = first_line.split()
  first_line = '\t'.join(first_line_split)+'\n'
  newopen.write(first_line)
  #print(first_line)

with open("result2_tmpOx.txt") as f:
  for line in f:
    if 'ID Element	El.' not in line:
      line_split = line.split()
      line = '\t'.join(line_split)+'\n'
      #print(line)
      newopen.write(line)
newopen.close()

input_file_path = 'result2_finalTmpOx.txt'
output_file_path = 'result2_finalOx.txt'

# Read the file and filter out empty lines
with open(input_file_path, 'r') as file:
    lines = file.readlines()
    non_empty_lines = [line.strip() for line in lines if line.strip()]

# Write non-empty lines back to the file
with open(output_file_path, 'w') as file:
    file.write('\n'.join(non_empty_lines))

os.remove(input_file_path)
os.remove('result2_tmpOx.txt')

print("completed OX")

completed


RESULT3.TXT con dati estratti di SD e DL

Extract X-Y coordinates

In [ ]:
# EXTRACT X and Y coodinates
file_in2 = open(fn, "r")
lines = file_in2.readlines()
names = []
Xs = []
Ys = []
Zs = []

for i, line in enumerate(lines):
    line_s = line.split()
    #print(line_s)
    if line_s:
        try:
            if(line_s[4] == "Comment"):
                # Keep track of the comment value
                name = line_s[6]
                names.append(name)
                # Just a spacing line
            elif(line_s[0] == "Stage"):
                # Keep track of the comment value
                x = line_s[3]
                Xs.append(x)
                y = line_s[5]
                Ys.append(y)
                z = line_s[7]
                Zs.append(z)
                # Just a spacing line
        except IndexError:
            pass

import pandas as pd
# Just a spacing line
print("Sample: ",len(names))
print("X: ", len(Xs))
print("Y: ", len(Ys))
print("Z: ", len(Zs))
out = pd.DataFrame(
    {'Sample': names,
     'X':Xs,
     'Y':Ys,
     'Z':Zs
     })
# Just a spacing line
#print(out)
out.to_csv('coordinate.csv', index=False)

Delete temporary *files*

In [ ]:

import os
file_path = 'result2_tmpSD.txt'
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File '{file_path}' has been deleted.")
else:
    print(f"File '{file_path}' does not exist.")

file_path = 'result2_final_tmpSD.txt'
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)
    print(f"File '{file_path}' has been deleted.")
else:
    print(f"File '{file_path}' does not exist.")



COORDINATE.CSV con sample name e coordinate X e Y

TUTTO DA **SCARICARE**